# **📓 03: Interfaz Web Interactiva con Streamlit**

Este notebook se encarga de lanzar la interfaz de usuario web para el proyecto VIREC. La aplicación permite a los usuarios interactuar con los modelos de clasificación entrenados.

**Flujo de Trabajo:**
1.  **Instalación:** Se instalan las librerías necesarias (`streamlit`, `pyngrok`, `pandas`).
2.  **Configuración:** Se monta Google Drive, se localiza la carpeta del proyecto y se lee el `registro_de_experimentos.csv` para listar los modelos disponibles.
3.  **Selección de Modelo:** El usuario selecciona interactivamente qué modelo desea cargar en la interfaz a través de un menú desplegable.
4.  **Escritura de Archivos:** El código de la aplicación (`app.py`, `style.css`, etc.) se escribe en el disco, inyectando dinámicamente la ruta al modelo seleccionado.
5.  **Lanzamiento:** Se inicia el servidor de Streamlit y se crea un túnel público con `ngrok` para hacer la aplicación accesible desde cualquier navegador.

In [ ]:
# ====================================================================================
# @title PASO 1: SETUP DEL ENTORNO
# ====================================================================================
!pip install -r requirements.txt -q

import streamlit as st
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import base64
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from google.colab import drive

print("✅ Librerías instaladas.")

### **Configuración y Selección de Modelo**
Esta celda conecta Google Drive, localiza el proyecto y lee el `registro_de_experimentos.csv` para encontrar todos los modelos que has entrenado. Por favor, selecciona del menú desplegable el experimento que deseas cargar en la interfaz web.

In [ ]:
# ====================================================================================
# @title PASO 2: CONFIGURACIÓN Y SELECCIÓN DE MODELO
# ====================================================================================

# --- 1. Montar Google Drive y buscar la ruta del proyecto ---
if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')
else:
    print("Google Drive ya está montado.")

NOMBRE_CARPETA_ANCLA = 'Proyecto_VIREC'
RUTA_BASE_PROYECTO = None
print(f"\nBuscando la carpeta ancla '{NOMBRE_CARPETA_ANCLA}'...")
for root, dirs, files in os.walk('/content/drive/MyDrive'):
    if NOMBRE_CARPETA_ANCLA in dirs:
        RUTA_BASE_PROYECTO = os.path.join(root, NOMBRE_CARPETA_ANCLA)
        break

if not RUTA_BASE_PROYECTO:
    raise FileNotFoundError(f"❌ ERROR CRÍTICO: No se pudo encontrar la carpeta '{NOMBRE_CARPETA_ANCLA}'.")
else:
    print(f"✅ ¡Proyecto encontrado! La ruta base es: {RUTA_BASE_PROYECTO}")
    
    # --- 2. Importar la configuración del usuario desde config.py ---
    import sys
    sys.path.append(RUTA_BASE_PROYECTO)
    try:
        # Importamos las variables que este notebook necesita
        from config import NGROK_AUTH_TOKEN
        print("✅ Configuración local importada desde config.py.")
    except ImportError:
        NGROK_AUTH_TOKEN = None # Definimos como None si no se encuentra
        print("⚠️ ADVERTENCIA: No se encontró 'config.py'. El token de Ngrok no se cargará.")

    # --- 3. Leer el Registro Centralizado de Experimentos ---
    ruta_registro = os.path.join(RUTA_BASE_PROYECTO, 'modelos_entrenados', 'registro_de_experimentos.csv')
    modelos_disponibles = []
    rutas_de_modelos = {}
    tamanos_de_modelos = {}

    if not os.path.exists(ruta_registro):
        print(f"❌ ADVERTENCIA: No se encontró el archivo de registro en '{ruta_registro}'.")
    else:
        df_registro = pd.read_csv(ruta_registro)
        if 'experiment_id' in df_registro.columns and 'ruta_modelo_guardado' in df_registro.columns:
            for index, row in df_registro.iterrows():
                display_name = f"{row['experiment_id']} (val_acc: {row['best_val_accuracy']})"
                modelos_disponibles.append(display_name)
                rutas_de_modelos[display_name] = row['ruta_modelo_guardado']
                tamanos_de_modelos[display_name] = int(row['img_size'])
            
            if not modelos_disponibles:
                 print("❌ ADVERTENCIA: El registro de experimentos está vacío.")
            else:
                dropdown_modelos = widgets.Dropdown(
                    options=modelos_disponibles,
                    description='Elige un experimento:',
                    style={'description_width': 'initial'},
                    layout={'width': 'max-content'}
                )
                print("\n👇 Por favor, selecciona el experimento a cargar en la interfaz:")
                display(dropdown_modelos)
        else:
            print("❌ ADVERTENCIA: El archivo de registro no tiene las columnas requeridas.")

    # --- 4. Guardar la configuración para la app ---
    # La app de Streamlit necesita saber la ruta base para encontrar el modelo
    ruta_config_txt = os.path.join(RUTA_BASE_PROYECTO, "config.txt")
    with open(ruta_config_txt, "w") as f:
        f.write(RUTA_BASE_PROYECTO)
    print(f"\n✅ Archivo de configuración para la app guardado en: {ruta_config_txt}")

### **Generación de Archivos de la Aplicación**
Esta celda crea la estructura de carpetas y escribe el contenido de la aplicación web (`app.py`, `style.css`) en el disco, inyectando dinámicamente la ruta al modelo que seleccionaste en el paso anterior.

In [ ]:
# ==================================================================================== 
# @title PASO 3: ESCRIBIR ARCHIVO DE LA APLICACIÓN (app.py)
# ==================================================================================== 

# --- 1. Definir rutas a los archivos estáticos y de la app ---
# Estas variables ya deberían estar definidas en la celda anterior
if 'RUTA_APP' in locals():
    RUTA_STATIC = os.path.join(RUTA_APP, 'static')
else:
    print("❌ ERROR: La variable 'RUTA_APP' no está definida. Ejecuta la celda de 'Crear Estructura de Archivos' primero.")
    # Detener si RUTA_APP no existe
    raise NameError("RUTA_APP no está definida.")

# --- 2. Verificar que los archivos estáticos existen ---
ruta_css = os.path.join(RUTA_STATIC, 'style.css')
ruta_logo = os.path.join(RUTA_STATIC, 'logo.png')

if not os.path.exists(ruta_css):
    print(f"❌ ADVERTENCIA: No se encontró el archivo 'style.css' en {RUTA_STATIC}.")
else:
    print("✅ Archivo 'style.css' encontrado.")
    
if not os.path.exists(ruta_logo):
    print(f"⚠️ ADVERTENCIA: No se encontró el archivo 'logo.png' en {RUTA_STATIC}.")
else:
    print("✅ Archivo 'logo.png' encontrado.")


# --- 3. Recuperar la selección del modelo ---
if 'dropdown_modelos' in locals() and 'rutas_de_modelos' in locals():
    seleccion_usuario = dropdown_modelos.value
    RUTA_RELATIVA_MODELO_SELECCIONADO = rutas_de_modelos[seleccion_usuario]
    IMG_SIZE_SELECCIONADO = tamanos_de_modelos[seleccion_usuario]
    print(f"✅ Modelo seleccionado para la app: '{seleccion_usuario}'")
else:
    RUTA_RELATIVA_MODELO_SELECCIONADO = "modelo_no_encontrado.keras"
    print(f"⚠️ Usando ruta de modelo por defecto.")


# --- 4. Definir contenido del archivo app.py ---
contenido_app_py = f"""
import streamlit as st
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import base64

# --- FUNCIONES DE UTILIDAD ---
def load_local_asset(file_path):
    with open(file_path, "r") as f:
        return f.read()

def get_image_as_base64(file_path):
    try:
        with open(file_path, "rb") as f:
            data = f.read()
        return base64.b64encode(data).decode()
    except IOError:
        return None

# ====================================================================================
#               CARGA DINÁMICA DEL MODELO (RNF5)
# ====================================================================================
@st.cache_resource
def cargar_modelo(ruta_base_proyecto):
    if not ruta_base_proyecto:
        st.error("Error: No se pudo determinar la ruta base del proyecto.")
        return None

    # La ruta relativa del modelo ahora se inserta dinámicamente
    RUTA_RELATIVA_MODELO = '{RUTA_RELATIVA_MODELO_SELECCIONADO}'
    ruta_modelo_completa = os.path.join(ruta_base_proyecto, RUTA_RELATIVA_MODELO)

    if os.path.exists(ruta_modelo_completa):
        modelo = tf.keras.models.load_model(ruta_modelo_completa)
    else:
        st.warning("Advertencia: Modelo no encontrado. Usando modo de demostración.")
        st.code(f"Buscando en: {{ruta_modelo_completa}}")
        modelo = None
    return modelo

# ====================================================================================
#                             LÓGICA DE PREDICCIÓN
# ====================================================================================
def predecir(modelo, imagen_pil):
    if modelo is None:
        prob = np.random.rand()
        return ("Reciclable" if prob > 0.5 else "No Reciclable"), prob

    IMG_SIZE_APP = ({IMG_SIZE_SELECCIONADO}, {IMG_SIZE_SELECCIONADO})
    img = imagen_pil.resize(IMG_SIZE_APP)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    prediccion = modelo.predict(img_array)
    probabilidad = prediccion[0][0]

    clase = "Reciclable" if probabilidad > 0.5 else "No Reciclable"
    confianza = probabilidad if probabilidad > 0.5 else 1 - probabilidad
    return clase, confianza

# ====================================================================================
#                           INTERFAZ PRINCIPAL DE LA APP
# ====================================================================================
def main():
    st.set_page_config(page_title="VIREC", page_icon="♻️", layout="centered")
    css_content = load_local_asset("static/style.css")
    st.markdown(f"<style>{{css_content}}</style>", unsafe_allow_html=True)
    logo_base64 = get_image_as_base64("static/logo.png")

    ruta_base = None
    if os.path.exists("../../config.txt"):
        with open("../../config.txt", "r") as f:
            ruta_base = f.read().strip()
    modelo_cargado = cargar_modelo(ruta_base)

    with st.sidebar:
        if logo_base64:
            st.markdown(f'<div style="text-align: center;"><img src="data:image/png;base64,{{logo_base64}}" width="100"></div>', unsafe_allow_html=True)
        st.title("Acerca de VIREC")
        st.info(
            "**VIREC (Visión Inteligente de Reciclaje)** es un prototipo que utiliza "
            "IA para clasificar residuos. Este proyecto fue desarrollado como parte "
            "del bootcamp de IA Nivel exploratorio de TalentoTECH."
        )

    st.title("♻️ Clasificador Inteligente de Residuos")
    st.header("Sube una imagen y descubre si es reciclable")

    fuente_imagen = st.radio("Elige una fuente:", ["Subir Archivo", "Usar Cámara"], horizontal=True)
    imagen_subida = st.file_uploader("Arrastra o selecciona una imagen", type=["jpg", "jpeg", "png"]) if fuente_imagen == "Subir Archivo" else st.camera_input("Toma una foto de un residuo")

    if imagen_subida:
        imagen = Image.open(imagen_subida)

        col1, col2, col3 = st.columns([1, 2, 1])
        with col2:
            st.image(imagen, caption="Imagen cargada", use_container_width=True)

        with st.spinner('Analizando...'):
            clase, confianza = predecir(modelo_cargado, imagen)

        clase_css = "reciclable" if clase == "Reciclable" else "no-reciclable"
        texto_clase_css = "reciclable-text" if clase == "Reciclable" else "no-reciclable-text"

        st.markdown(f'''
        <div class="result-card {{clase_css}}">
            <p class="result-text {{texto_clase_css}}">{{clase}}</p>
            <p class="confidence-text">Confianza: {{confianza:.2%}}</p>
        </div>
        ''', unsafe_allow_html=True)

if __name__ == "__main__":
    main()
"""

# --- 5. Escribir el archivo app.py en el disco ---
if 'RUTA_APP_PY' in locals():
    try:
        with open(RUTA_APP_PY, "w") as f:
            f.write(contenido_app_py)
        print(f"✅ Archivo app.py escrito exitosamente en: {RUTA_APP_PY}")
    except Exception as e:
        print(f"❌ ERROR al escribir el archivo app.py: {e}")
else:
    print("❌ ERROR: La variable 'RUTA_APP_PY' no está definida. Ejecuta la celda de 'Crear Estructura de Archivos' primero.")

### **Lanzamiento de la Aplicación Web**
Esta celda final lanza la aplicación de Streamlit y crea un túnel público con ngrok.

**Importante:** Antes de ejecutar, asegúrate de haber creado el archivo `config.py` en la raíz de tu proyecto (`Proyecto_VIREC/`) a partir de la plantilla `config.py.template` y de haber añadido tu token de Ngrok.

In [ ]:
# ====================================================================================
# @title PASO 4: LANZAR LA APLICACIÓN WEB
# ====================================================================================
from pyngrok import ngrok
import time

# --- 1. Importar el token de ngrok desde el archivo de configuración ---
import sys
sys.path.append(RUTA_BASE_PROYECTO)
try:
    from config import NGROK_AUTH_TOKEN
    print("✅ Token de Ngrok importado desde config.py.")
except ImportError:
    NGROK_AUTH_TOKEN = None
    print("❌ ERROR: No se pudo importar NGROK_AUTH_TOKEN desde 'config.py'.")

# --- 2. Ejecutar la aplicación ---
if NGROK_AUTH_TOKEN:
    # Limpiar sesiones anteriores
    print("\n🧹 Limpiando sesiones anteriores...")
    os.system("killall streamlit")
    ngrok.kill()
    
    # Moverse al directorio de la app para que las rutas relativas funcionen
    %cd {RUTA_APP}
    
    # Lanzar la app en segundo plano
    print("🚀 Lanzando la aplicación...")
    get_ipython().system("nohup streamlit run app.py --server.headless true &")
    
    print("   -> Esperando 5 segundos para que el servidor se inicie...")
    time.sleep(5)
    
    # Conectar ngrok
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    public_url = ngrok.connect(8501)
    print("\n" + "="*60)
    print("🎉 ¡Tu aplicación está en línea!")
    print(f"🔗 Abre esta URL en tu navegador: {public_url}")
    print("="*60)
    
    %cd /content
else:
    print("   Por favor, crea tu archivo 'config.py' y añade tu token de Ngrok para continuar.")